# Importing libraries

First of all, we need to import some external libraries and our own helper files:

In [1]:
print("Importing libraries")
# system libraries
import os
import logging

# third party libraries
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

# Own helper files
from ml_webinar import preprocessing, evaluate, plots

logging.basicConfig(level=logging.INFO)
print("Done!")

Importing libraries
Done!


# File input

Next, we indicate where our data can be found, several variables for the algorithm, and the correct labels for our data. Since we use this script for several demo's with different datasets, we indicate what the variable DATA_FOLDER should be for each demo.

In [2]:
#Quiz 2: 'data/data_rotation_orig1'
#Quiz 3: 'data/data_rotation_orig2'
#Quiz 4: 'data/data_rotation_orig3'
DATA_FOLDER = 'data/data_rotation_orig3'
IMG_PIX = 256

if DATA_FOLDER == "data/data_colors":
    CLASS_DICT = {
        "c_black": {"index": 0, "name": "Black"},
        "c_blue": {"index": 1, "name": "Blue"},
        "c_red": {"index": 2, "name": "Red"},
        "c_white": {"index": 3, "name": "White"}
    }
else:
    CLASS_DICT = {
        "F": {"index": 0, 'name': "Front"},
        "B": {"index": 1, 'name': "Back"}, 
        "R": {"index": 2, 'name': "Right"},
        "L": {"index": 3, 'name': "Left"},
        "FR": {"index": 4, 'name': "Front Right"},
        "FL": {"index": 5, 'name': "Front Left"},
    }
    
    
EPOCHS = {
    'data/data_rotation_orig1': 5,
    'data/data_rotation_orig2': 5,
    'data/data_rotation_orig3': 20,
    'data/data_colors': 10
}.get(DATA_FOLDER)

if DATA_FOLDER in ['data/data_rotation_orig3', 'data/data_colors']:
    WIDTH_SHIFT, HEIGHT_SHIFT = 0.1, 0.1
else:
    WIDTH_SHIFT, HEIGHT_SHIFT = 0, 0
    
SEED = 1234
np.random.seed(seed=SEED)
tf.random.set_seed(SEED)

# Creating a training set and test set

The pictures in the dataset need to be reformatted before they can be used: we rescale the pixel values, indicate we want the RGB color model, and assign the images to either the training set or test set.

In [3]:
def make_generator(phase, width_shift=0, height_shift=0):
    image_gen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                            width_shift_range=height_shift,
                                                            height_shift_range=width_shift)
    return image_gen.flow_from_directory(directory=os.path.join(DATA_FOLDER, phase),
                                                     batch_size=32,
                                                     shuffle=True,
                                                     target_size=(IMG_PIX, IMG_PIX),
                                                     classes=list(CLASS_DICT.keys()),
                                                     class_mode='sparse',
                                                     color_mode="rgb",
                                                     seed=SEED)

Executing this function for the test set and training set shows us how many images end up in each set.

In [4]:
train_data_gen = make_generator('train', width_shift=WIDTH_SHIFT, height_shift=HEIGHT_SHIFT)
test_data_gen = make_generator('test')

Found 1473 images belonging to 6 classes.
Found 134 images belonging to 6 classes.


# Building the model

Once the data preparation is done, we can start building the model. In this example, we use a more advanced Neural Network with seven middle layers.

In [5]:
cv_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_PIX, IMG_PIX, 3), name='block1_conv1'),
    keras.layers.MaxPooling2D((2, 2), name='block1_maxpool'),
    keras.layers.Conv2D(64, (3, 3), activation='relu', name='block2_conv1'),
    keras.layers.MaxPooling2D((2, 2), name='block2_maxpool'),
    keras.layers.Conv2D(32, (3, 3), activation='relu', name='block3_pool'),
    keras.layers.Flatten(name='flatten_6'),
    keras.layers.Dense(64, activation='relu', name='fc1'),
    keras.layers.Dense(len(CLASS_DICT.keys()), name='fc2'),
])

print("Compiling the model")
cv_model.compile(optimizer='adam', 
                 loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
                 metrics=['accuracy'])
print("Done!")

Compiling the model
Done!


We can ask TensorFlow to give us some more information about the model we just created. In the webinar, however, we already show this in the slides.

In [6]:
#cv_model.summary()
#keras.utils.plot_model(cv_model, to_file='reports/model_2.png', show_shapes=True)

# Training the model

Now, we are finally at the point where we can feed the model data to teach it to classify our images correctly.

In [ ]:
cv_model.fit(train_data_gen, epochs=EPOCHS)

Epoch 1/20
47/47 [==============================] - 105s 2s/step - loss: 2.0219 - accuracy: 0.2396
Epoch 2/20
47/47 [==============================] - 96s 2s/step - loss: 1.6764 - accuracy: 0.3014
Epoch 3/20
47/47 [==============================] - 94s 2s/step - loss: 1.6228 - accuracy: 0.3157
Epoch 4/20
47/47 [==============================] - 97s 2s/step - loss: 1.5144 - accuracy: 0.3659
Epoch 5/20
47/47 [==============================] - 93s 2s/step - loss: 1.3379 - accuracy: 0.4474
Epoch 6/20
39/47 [=======================>......] - ETA: 16s - loss: 1.2632 - accuracy: 0.4610

After we let the model train on the training data several times, we can add the final output layer which allows us to make easier predictions:

In [ ]:
probability_model = tf.keras.Sequential([cv_model, tf.keras.layers.Softmax(name='softmax_9')])

Just like before, we can ask TensorFlow to give us some more information about the (now trained) model. This can be useful for debugging, but is irrelevant for the webinar demo.

In [ ]:
#keras.utils.plot_model(probability_model, to_file='reports/model_2prob.png', show_shapes=True)

Now that we have a trained model, we can see how well the model performs when we make it pretend the training set is the test set. This allows us to discover problems such as overfitting.

In [ ]:
train_images, train_labels = evaluate.get_images('train', DATA_FOLDER, CLASS_DICT, IMG_PIX)
insample_classification = probability_model.predict(train_images)
df_confusion_train = evaluate.make_confusion_matrix(train_labels, insample_classification, CLASS_DICT)
df_confusion_train

The classification performance a model gets on the data that is used to train the model is called the train accuracy:

In [ ]:
train_accuracy = evaluate.get_accuracy(df_confusion_train)
print(' train accuracy %3.2f' % train_accuracy)

# Making predictions

With all the training out of the way, we can feed the Neural Network test data. The algorithm has not seen these images before, but it can use the correct labels to determine the overall accuracy of its predictions.

In [ ]:
cv_model.evaluate(test_data_gen)
test_images, test_labels = evaluate.get_images('test', DATA_FOLDER, CLASS_DICT, IMG_PIX)
predictions = probability_model.predict(test_images)

df_confusion_test = evaluate.make_confusion_matrix(test_labels, predictions, CLASS_DICT)
df_confusion_test

The classification performance a model gets on the test set data is called the test accuracy:

In [ ]:
test_accuracy = evaluate.get_accuracy(df_confusion_test)
print(' test accuracy %3.2f' % test_accuracy)

# Graphing the predictions

In [ ]:
from random import randrange
idx_img = randrange(len(predictions))
plots.plot_image_and_predictions(test_images[idx_img], test_labels[idx_img], predictions[idx_img], CLASS_DICT)

In the plot below, Blue means a correct prediction, and Red means an incorrect prediction. The label below each image shows what the model predicted, and what the actual label is. Can you spot some improvements we can make to the model or dataset?

In [ ]:
plots.plot_multiple_images_and_predictions(predictions, test_labels, test_images, CLASS_DICT)

# Acknowledgements

In [ ]:
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.